In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')

In [2]:
#This will create a Collection/Database called department if it doesn't already exist

coll_department = client['department']

department = coll_department.department

In [3]:
#Inserts department names and heads data
department.insert_many([{"dep_name": "IT", "DepartmentHead":"Jason"},
                            {"dep_name": "Admin", "DepartmentHead":"Nial"},
                            {"dep_name": "Accounts", "DepartmentHead":"Harris"}])

##In order to create the data we will pandas dataframe. A Data frame is a two-dimensional data structure, i.e., data is aligned in a tabular fashion in rows and columns. Pandas DataFrame consists of three principal components, the data, rows, and columns.

In [4]:
#This will create a Collection/Database called employees if it doesn't already exist
import pandas as pd

db = client['employees']

df_employees = pd.DataFrame(list(db.employees.find()))


In [5]:
df_employees.head()

,_id,name,Department,Salary
0,60ee97e6176167638660b0f3,Jessica,IT,6000
1,60ee97e6176167638660b0f4,Joseph,IT,7000
2,60ee97e6176167638660b0f5,Alex,Accounts,5000
3,60ee97e6176167638660b0f6,Julie,IT,3000
4,60ee97e6176167638660b0f7,James,Admin,8000


In [6]:
df_department = pd.DataFrame(list(department.find()))

In [7]:
df_department.head()

,_id,dep_name,DepartmentHead
0,60ee9c4b256841cde70955d7,IT,Jason
1,60ee9c4b256841cde70955d8,Admin,Nial
2,60ee9c4b256841cde70955d9,Accounts,Harris


### Left Join: 

Left join uses only keys from left frame, similar to a SQL left outer join

In [8]:
df_employees.merge(df_department,left_on="Department",right_on="dep_name",how="left")

,_id_x,name,Department,Salary,_id_y,dep_name,DepartmentHead
0,60ee97e6176167638660b0f3,Jessica,IT,6000,60ee9c4b256841cde70955d7,IT,Jason
1,60ee97e6176167638660b0f4,Joseph,IT,7000,60ee9c4b256841cde70955d7,IT,Jason
2,60ee97e6176167638660b0f5,Alex,Accounts,5000,60ee9c4b256841cde70955d9,Accounts,Harris
3,60ee97e6176167638660b0f6,Julie,IT,3000,60ee9c4b256841cde70955d7,IT,Jason
4,60ee97e6176167638660b0f7,James,Admin,8000,60ee9c4b256841cde70955d8,Admin,Nial
5,60ee97e6176167638660b0f8,Jacob,Admin,9000,60ee9c4b256841cde70955d8,Admin,Nial
6,60ee97e6176167638660b0f9,Kevin,IT,6500,60ee9c4b256841cde70955d7,IT,Jason


### Inner Join: 

Use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

In [9]:
df_employees.merge(df_department,left_on="Department",right_on="dep_name",how="inner")

,_id_x,name,Department,Salary,_id_y,dep_name,DepartmentHead
0,60ee97e6176167638660b0f3,Jessica,IT,6000,60ee9c4b256841cde70955d7,IT,Jason
1,60ee97e6176167638660b0f4,Joseph,IT,7000,60ee9c4b256841cde70955d7,IT,Jason
2,60ee97e6176167638660b0f6,Julie,IT,3000,60ee9c4b256841cde70955d7,IT,Jason
3,60ee97e6176167638660b0f9,Kevin,IT,6500,60ee9c4b256841cde70955d7,IT,Jason
4,60ee97e6176167638660b0f5,Alex,Accounts,5000,60ee9c4b256841cde70955d9,Accounts,Harris
5,60ee97e6176167638660b0f7,James,Admin,8000,60ee9c4b256841cde70955d8,Admin,Nial
6,60ee97e6176167638660b0f8,Jacob,Admin,9000,60ee9c4b256841cde70955d8,Admin,Nial


### To get the total number of records in the collection

In [10]:
pipeline = [
     {"$group": {"_id": None,"Count": {"$sum": 1}}}]
grp_employees = db.employees.aggregate(pipeline)

In [11]:
for employee in grp_employees:
  print(employee)

{'_id': None, 'Count': 7}


### To group by department and get the total salary for each department

In [12]:
pipeline = [
     {"$group": {"_id": "$Department","Salary": {"$sum": "$Salary"}}}]
grp_employees = db.employees.aggregate(pipeline)

In [13]:
for employee in grp_employees:
  print(employee)

{'_id': 'Admin', 'Salary': 17000}
{'_id': 'Accounts', 'Salary': 5000}
{'_id': 'IT', 'Salary': 22500}


### To group by department and get the average salary for each department

In [ ]:
pipeline = [
     {"$group": {"_id": "$Department","Salary": {"$avg": "$Salary"}}}]
grp_employees = db.employees.aggregate(pipeline)


In [14]:
for employee in grp_employees:
  print(employee['_id'],"\t Average Salary ",employee['Salary'])

### To get the employees with the lowest salary

In [15]:
pipeline = [
     {"$group": {"_id": None ,"Minimum Salary": {"$min": "$Salary"}}}]
grp_employees = db.employees.aggregate(pipeline)


In [16]:
for employee in grp_employees:
    for spec_emp in db.employees.find({"Salary":employee['Minimum Salary']}):
        print("Name ",spec_emp['name'],"\nSalary ",spec_emp['Salary'])

Name  Julie 
Salary  3000
